In [ ]:
#Kacper Klocek Data Analytics project

In [5]:
#Data Cleaning
import pandas as pd
data = pd.read_csv('Dataset_files/prosecutionProject.csv')

# Step 3: Remove the "Short narrative" column
#Removed the "Short narrative" column as it servers no use in data analysis
data_cleaned = data.drop(columns=['Short narrative'])


missing_values_summary = data_cleaned.isnull().sum()
columns_to_drop = missing_values_summary[missing_values_summary > (0.5 * len(data_cleaned))].index
#Dropped columns with over 50% missing data
data_cleaned = data_cleaned.drop(columns=columns_to_drop)

#Converted Data to datatime
data_cleaned['Date'] = pd.to_datetime(data_cleaned['Date'], errors='coerce')

# Display cleaned dataset
#print(data_cleaned.head())

#Saved the cleaned data, which will be used for the remainder of the project
data_cleaned.to_csv('Dataset_files/cleaned_dataset.csv', index=False)
